In this assigment we are building a regression tree. Boosting is like playing golf. First you hit quite hard. Then you try multiple times to come as near to the hole as possible. Don't make it too deep to avoid overfitting.  


In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb



Download the dataset on Uˇcilnica "wine-quality.csv"

In [4]:
data = pd.read_csv('wine_quality.csv')

In [ ]:

data['quality'] = data['quality'].map({'low': 0, 'high': 1})

In [ ]:

data['alcohol_level'] = data['alcohol_level'].map({'low': 0, 'high': 1})


print(data[['quality', 'alcohol_level']].head())


X = data.drop(['quality'], axis=1)
y = data['quality']


X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42
)  # 0.25 x 0.8 = 0.2


   quality  alcohol_level
0        0              0
1        0              0
2        0              1
3        0              0
4        0              0


Write the code for gradient boosting of trees to solve a binary classification problem
from scratch.Use the Scikit-learn regression tree implementation. The trees have to be weak learners,
so their depth has to be set to a small value. You can choose their final depth arbitrar-
ily. Think about overfitting.

. Number of estimators is basically the number of boosting stages to be performed by the model

The learning rate determines the step size at each iteration while moving toward a minimum of a loss function

In [ ]:

def gradient_boosting(X_train, y_train, X_val, y_val, learning_rate, n_estimators, max_depth):
    
    y_pred = np.full(y_train.shape, y_train.mean())
    models = []
    for _ in range(n_estimators):
        residuals = y_train - y_pred
        tree = DecisionTreeRegressor(max_depth=max_depth)
        tree.fit(X_train, residuals)
        update = tree.predict(X_train)
        y_pred += learning_rate * update
        models.append(tree)
    
    y_val_pred = np.full(y_val.shape, y_train.mean())
    for tree in models:
        y_val_pred += learning_rate * tree.predict(X_val)
    y_val_pred_class = (y_val_pred >= 0.5).astype(int)
    accuracy = accuracy_score(y_val, y_val_pred_class)
    return accuracy

Test different tree depths. The trees have to be weak learners.
Test different learning rates. 
Test different numbers of trees that are built during gradient boosting and comment


In [ ]:

learning_rates = [0.1, 0.05, 0.01]
max_depths = [1, 2, 3]
n_estimators_list = [50, 100, 200]

best_accuracy = 0
best_params = {}

for lr in learning_rates:
    for depth in max_depths:
        for n_estimators in n_estimators_list:
            accuracy = gradient_boosting(X_train, y_train, X_val, y_val, lr, n_estimators, depth)
            print(f'Learning Rate: {lr}, Max Depth: {depth}, N Estimators: {n_estimators}, Accuracy: {accuracy}')
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'learning_rate': lr, 'max_depth': depth, 'n_estimators': n_estimators}

print('Best Parameters:', best_params)

Learning Rate: 0.1, Max Depth: 1, N Estimators: 50, Accuracy: 0.7836734693877551
Learning Rate: 0.1, Max Depth: 1, N Estimators: 100, Accuracy: 0.7908163265306123
Learning Rate: 0.1, Max Depth: 1, N Estimators: 200, Accuracy: 0.8
Learning Rate: 0.1, Max Depth: 2, N Estimators: 50, Accuracy: 0.8071428571428572
Learning Rate: 0.1, Max Depth: 2, N Estimators: 100, Accuracy: 0.8163265306122449
Learning Rate: 0.1, Max Depth: 2, N Estimators: 200, Accuracy: 0.8326530612244898
Learning Rate: 0.1, Max Depth: 3, N Estimators: 50, Accuracy: 0.8214285714285714
Learning Rate: 0.1, Max Depth: 3, N Estimators: 100, Accuracy: 0.8387755102040816
Learning Rate: 0.1, Max Depth: 3, N Estimators: 200, Accuracy: 0.8469387755102041
Learning Rate: 0.05, Max Depth: 1, N Estimators: 50, Accuracy: 0.7826530612244897
Learning Rate: 0.05, Max Depth: 1, N Estimators: 100, Accuracy: 0.7836734693877551
Learning Rate: 0.05, Max Depth: 1, N Estimators: 200, Accuracy: 0.7908163265306123
Learning Rate: 0.05, Max Depth: 

Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}

What is a good learning rate? A good learning rate is 0.1



In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    random_state=42
)
gbc.fit(X_train, y_train)
gbc_pred = gbc.predict(X_val)
gbc_accuracy = accuracy_score(y_val, gbc_pred)
print('GradientBoostingClassifier Accuracy:', gbc_accuracy)

GradientBoostingClassifier Accuracy: 0.8520408163265306



Compare the results from your implementation with the "GradientBoostingClassi-
fier" classifier implemented in Scikit-learn.



GradientBoostingClassifier Accuracy: 0.852

Moja implementacija -> Accuracy: 0.847 Learning Rate: 0.1, Max Depth: 3, N Estimators: 200, 

In [13]:
from sklearn.model_selection import GridSearchCV


xgb_reg = xgb.XGBRegressor(
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    objective='reg:squarederror',
    random_state=42
)
xgb_reg.fit(X_train, y_train)
xgb_pred = xgb_reg.predict(X_val)
xgb_mse = mean_squared_error(y_val, xgb_pred)
xgb_r2 = r2_score(y_val, xgb_pred)
print('XGBoost Regressor MSE:', xgb_mse)
print('XGBoost Regressor R2:', xgb_r2)
param_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [1, 2, 3],
    'n_estimators': [50, 100, 200,500]
}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)
print("Lowest RMSE found: ", (-grid_search.best_score_)**0.5)

XGBoost Regressor MSE: 0.10630660945269882
XGBoost Regressor R2: 0.3750613331794739
Best parameters found:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
Lowest RMSE found:  0.34236755904118854


Try modeling your data also with XGBoost and try to find the best set of hyperpa-
rameters.
When you perform hyperparameter tuning be careful to use a validation set


Best parameters found:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
